In [82]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [83]:
data =  pd.read_csv("/Users/krushna/Documents/CSV Files/Missing Data/Automate Indicator /train-4.csv")

In [84]:
data 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


How it works ? So, it tries every method on the data and columns we select and gives out the best method with the help of estimating model accuracy

In [85]:
# Dropping columns that are not necessary
data.drop(columns=['PassengerId','Name','Ticket', 'Cabin'], inplace= True)

In [86]:
X = data.drop(columns=['Survived'])
y = data['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [87]:
# Numerical Data Dealing 

In [88]:
Numerical_cols = ['Age','Fare']

In [89]:
# Create PipeLine -> Beacuse we are doig 2 things together 1- PreProcessing 2- Imputation
Numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [90]:
# Categorical Data Dealing 

In [91]:
Categorical_cols = ['Embarked','Sex']

In [92]:
# Create PipeLine -> Beacuse we are doig 2 things together 1- PreProcessing 2- Imputation
Categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore'))
])

In [93]:
# Mixing up Both Numerical and Ctegorical Changes 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Numerical_pipeline, Numerical_cols),
        ('cat',Categorical_pipeline, Categorical_cols)
    ]
)

In [94]:
# Now we are making a pipleine where w'll be doing ColumnTransfomer and another for Logistic Regression 
clf = Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('regression', LogisticRegression())]
)

In [95]:
from sklearn import set_config

set_config(display='diagram')
clf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('regression', LogisticRegression())])

In [96]:
# GridSearchCV
param_grid = {
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'preprocessor__cat__imputer__strategy': ['most_frequent', 'constant'],
    'regression__C': [0.1, 1.0, 10, 100]
}

grid_search = GridSearchCV(clf, param_grid, cv=10)
grid_search.fit(X_train, y_train)

print(f"Best params:")
print(grid_search.best_params_)

Best params:
{'preprocessor__cat__imputer__strategy': 'most_frequent', 'preprocessor__num__imputer__strategy': 'mean', 'regression__C': 1.0}


In [97]:
print(f"Internal CV score: {grid_search.best_score_:.3f}")

Internal CV score: 0.788


In [98]:
# Creating a DataFrame from the grid search results
cv_results = pd.DataFrame(grid_search.cv_results_)

# Sorting the DataFrame by mean_test_score in descending order
cv_results = cv_results.sort_values("mean_test_score", ascending=False)

# Displaying relevant columns
cv_results[['param_regression__C', 'param_preprocessor__cat__imputer__strategy', 'param_preprocessor__num__imputer__strategy', 'mean_test_score']]


,param_regression__C,param_preprocessor__cat__imputer__strategy,param_preprocessor__num__imputer__strategy,mean_test_score
1,1.0,most_frequent,mean,0.787852
2,10,most_frequent,mean,0.787852
3,100,most_frequent,mean,0.787852
5,1.0,most_frequent,median,0.787852
6,10,most_frequent,median,0.787852
7,100,most_frequent,median,0.787852
9,1.0,constant,mean,0.787852
10,10,constant,mean,0.787852
11,100,constant,mean,0.787852
13,1.0,constant,median,0.787852
